In [1]:
import os
# set the current working directory. This is required by isaac.
# This cell should only run once.
os.chdir("../../../")
os.getcwd()

'/home/mrobotics2/.cache/bazel/_bazel_mrobotics2/36d61b35046788db1f2a67ff239dff81/execroot/com_nvidia_isaac_sdk/bazel-out/k8-opt/bin/apps/samples/manipulation/simple_joint_control.runfiles/com_nvidia_isaac_sdk'

In [2]:
from IPython.display import display
import json
import numpy as np

from packages.pyalice import Application, Codelet,Message, Composite
from packages.pyalice.gui.composite_widget import CompositeWidget

np.set_printoptions(precision=3)

# Sophia Left Arm Simulator using Composite Widget
---




Sophia Kinematic file import and get the joint,
Example - ['13','17','19','21']

In [3]:
# set kinematic file and get list of joints for sophia
kinematic_file = "/home/mrobotics2/Desktop/isaac/sdk/apps/assets/kinematic_trees/sophia_left_arm.kinematic.json"
# kinematic_file = "/home/mrobotics2/Desktop/isaac/sdk/apps/assets/kinematic_trees/one_motor.kinematic.json"
joints = []
with open(kinematic_file,'r') as fd:
    kt = json.load(fd)
    for link in kt['links']:
        if 'motor' in link and link['motor']['type'] != 'constant':
            joints.append(link['name'])
# joints = ['9','11','15','13','17','19','21']
print(joints)

['9', '11', '13', '15', '17', '19', '21']



1. This Codelet Receives a Message as Command and sends command to Dynamixel Drivers
2. 

In [4]:

class DriverInputControl(Codelet):
    def start(self):
        self.rx = self.isaac_proto_rx("CompositeProto", "command") # received joint state
#         self.tx = self.isaac_proto_tx("StateProto", "state")
        
        self.tx_106 = self.isaac_proto_tx("StateProto", "command_106")
        self.tx_64 = self.isaac_proto_tx("StateProto", "command_64")
        self.tx_430 = self.isaac_proto_tx("StateProto", "command_430")
        
        
        self.tick_periodically(0.2)

    def tick(self):
        state_msg = self.rx.message
        if state_msg is None:
            return "Received None in DriverInputControl"
        
        # Get the Value of tensor and send value further, Supposed to get values for 4 joints
        # Hence a total of 8 values in a tensor list.
        angles_list_dynamixel = state_msg.tensor
        print("Driver Position List {}".format(angles_list_dynamixel))
        print('Checking Size of tensor ')
        print('Message Received from Composite Proto', state_msg.tensor)
        #Convert List
        
        
        tx_message_106 = self.tx_106.init()
        tx_message_64  = self.tx_64.init()
        tx_message_430 = self.tx_430.init()
        
        #Adding to Proto
        tx_message_106.proto.pack.elementType = 3  # Float64
        tx_message_106.proto.pack.sizes = [1, 1, 2]
        tx_message_106.proto.pack.scanlineStride = 0
        tx_message_106.proto.pack.dataBufferIndex = 0
        tx_message_106.proto.schema = "StateProto"
        
        tx_message_64.proto.pack.elementType = 3  # Float64
        tx_message_64.proto.pack.sizes = [1, 1, 1]
        tx_message_64.proto.pack.scanlineStride = 0
        tx_message_64.proto.pack.dataBufferIndex = 0
        tx_message_64.proto.schema = "StateProto"
        
        tx_message_430.proto.pack.elementType = 3  # Float64
        tx_message_430.proto.pack.sizes = [1, 1, 4]
        tx_message_430.proto.pack.scanlineStride = 0
        tx_message_430.proto.pack.dataBufferIndex = 0
        tx_message_430.proto.schema = "StateProto"
        
        #Fill buffer With Values
        buffer_106 = np.empty([1, 1, 2], dtype=np.dtype('float64'))
        buffer_64  = np.empty([1, 1, 1], dtype=np.dtype('float64'))
        buffer_430 = np.empty([1, 1, 4], dtype=np.dtype('float64'))
        from itertools import chain
        buffer_106[0][0][0] = angles_list_dynamixel[0]      # 9
        buffer_106[0][0][1] = angles_list_dynamixel[1]      # 11
        
        buffer_64[0][0][0]  = angles_list_dynamixel[2]      # 15
        
        buffer_430[0][0][0] = angles_list_dynamixel[3]      # 13  - [750-3480]
        buffer_430[0][0][1] = angles_list_dynamixel[4]      # 17  - [198-3859]
        buffer_430[0][0][2] = angles_list_dynamixel[5]      # 19  - [1643-2297]
        buffer_430[0][0][3] = angles_list_dynamixel[6]      # 21  - [1783-2383]

        
        '''
          Set Buffer
        '''
        tx_message_106.buffers = [buffer_106]
        tx_message_64.buffers  = [buffer_64]
        tx_message_430.buffers = [buffer_430]
        
        '''Publish'''
        self.tx_106.publish()
        self.tx_64.publish()
        self.tx_430.publish()
        

        '''
            1. Adding 3 Tx message for each motor.
            2. For each attach the buffer
        '''

In [5]:
# A Python codelet for joint control through widget
import numpy as np
from itertools import chain
class DriverOutputControl(Codelet):
    def start(self):
        '''
            Will receive three state from three different dynamixel components one for each type.
            Three RX as STATE 
        '''
        self.rx_106 = self.isaac_proto_rx("StateProto", "state_106")
        self.rx_64  = self.isaac_proto_rx("StateProto", "state_64")
        self.rx_430 = self.isaac_proto_rx("StateProto", "state_430")
        self.tx = self.isaac_proto_tx("CompositeProto", "state")
#         self._widget = CompositeWidget(self.config.joints, self.config.measure, self.config.limits)
        self.tick_periodically(0.2)
        print("Inside START of Driver OUTPUT")

    def tick(self):
        print('Inside TICK Sophia Output')
        state_msg_106 = self.rx_106.message  # MessageReader 106
        state_msg_64  = self.rx_64.message   # MessageReader 64
        state_msg_430 = self.rx_430.message  # MessageReader 430
        
        if state_msg_106 is None:
            return "106 NOT Received"
        if state_msg_64 is None:
            return "64 NOT Received"
        if state_msg_430 is None:
            return "430 NOT Received"
        
        if(True):
            print('__  106, 64 and 430 Message received with following details  __')
#             print('State Proto Pack  {}'.format(state_msg.proto.pack))
#             print('Schema {}'.format(state_msg.proto.schema))
#             print('Data {}'.format(state_msg.proto.data))
            print("Tensor 106 {}".format(state_msg_106.tensor)) #3D Tensor
            print("Tensor 64  {}".format(state_msg_64.tensor))
            print("Tensor 430 {}".format(state_msg_430.tensor))
            print('\t__END__')
            
        # Making Composite Message

        measure = "position"
        quantities = [[x, measure, 1] for x in joints]
#         print('Quantities are : \n', quantities)
        
#         print('Tensor Value from State Proto of ARM is ', state_msg.tensor[0][0],' and type  : ',type(state_msg.tensor))
        '''
            Combinig all tensor to one
        '''
        print("CHECK 1")
#         appended_tensor = list(chain(state_msg_106.tensor[0][0] ,state_msg_64.tensor[0][0],state_msg_430.tensor[0][0]))
        appended_tensor = np.concatenate((state_msg_106.tensor[0][0],state_msg_64.tensor[0][0],state_msg_430.tensor[0][0]))
        print("Tensor Combined Looks Like",appended_tensor)
        print("CHECK 2")
        #values = state_msg.tensor[0][0] #Send entire array
        values = appended_tensor #Send entire array
        
        comp_msg = Composite.create_composite_message(quantities, values)
#         print('Proto of  Message is : ',comp_msg.proto)
#         print('Tensor of Message :: ', comp_msg.proto.tensor)
        
        
        #Send Message out
        tx_message = self.tx.init()
        print("CHECK 3")
        tx_message.proto = comp_msg.proto
        tx_message.buffers = comp_msg.buffers
        
        self.tx.publish()
                

In [6]:
# A Python codelet for joint control through widget
class JointControl(Codelet):
    def start(self):
        print('\n --Inside Joint Controller-- \n')
        self.rx = self.isaac_proto_rx("CompositeProto", "command")
        self.tx = self.isaac_proto_tx("CompositeProto", "state")
        print('------------------------------  Step IN   ')
        self._widget = CompositeWidget(self.config.joints, self.config.measure, self.config.limits)
        print('------------------------------  Step OUT  ')
        if self._widget is None:
            report_failure("Cannot create valid widget")
            return "Cannot Create Valid Widget"
        display(self._widget.panel)
        self.tick_periodically(0.1)

    def tick(self):
        state_msg = self.rx.message
        
        if state_msg is None:
            return "NOTHING IS BEING RECEIVED BY DRIVER SOPHIA OUTPUT"
        
        print('Length of Quantities',len(state_msg.proto.quantities))
        print('Length of Tensor : ',len(state_msg.tensor))
        print('Quantities of ')
        self._widget.composite = state_msg
        
        self.tx._msg = self._widget.composite
        if self.tx._msg is not None:
            self.tx.publish()


In [7]:
'''
MAIN APPLICATION
'''


app = Application(name="simple_joint_control_lqr_sophia_larm")

# Load Applications
app.load(filename="/home/mrobotics2/Desktop/isaac/sdk/packages/planner/apps/multi_joint_lqr_control.subgraph.json", prefix="lqr")
lqr_interface = app.nodes["lqr.subgraph"]["interface"]

# # LQR configs
app.nodes["lqr.kinematic_tree"]["KinematicTree"].config.kinematic_file = kinematic_file
lqr_planner = app.nodes["lqr.local_plan"]["MultiJointLqrPlanner"]
lqr_planner.config.speed_min = [-1.0] * len(joints)
lqr_planner.config.speed_max = [1.0] * len(joints)
lqr_planner.config.acceleration_min = [-1.0] * len(joints)
lqr_planner.config.acceleration_max = [1.0] * len(joints)



# add pycodelet JointControl for arm control
joint_commander = app.add("command_generator").add(JointControl)
joint_commander.config.joints = joints
joint_commander.config.limits = [[480,2700],[480,2700],[740,3580],[780,3400],[198,3859],[1643,2297],[1783,2383]]
joint_commander.config.measure = "position"


#Adding nodes to Application
driver_sophia_output =  app.nodes["lqr.driver_sophia_output"]["DriverOutputControl"]
driver_sophia_input =  app.nodes["lqr.driver_sophia_input"]["DriverInputControl"]

app.nodes["lqr.driver_sophia_input"].add(name="DriverInputControl", ctype=DriverInputControl)
app.nodes["lqr.driver_sophia_output"].add(name="DriverOutputControl", ctype=DriverOutputControl)


# Edges
app.connect(driver_sophia_output, "state", joint_commander, "command")  # driver_output ----> joint_commander
app.connect(joint_commander, "state", driver_sophia_input, "command")  # driver_output ----> joint_commander



app.start()
#Make Dynamixel Work

2021-03-09 11:03:45,353 DEBUG Binding PyCodelet command_generator/PyCodelet
2021-03-09 11:03:45,354 DEBUG Binding PyCodelet lqr.driver_sophia_input/DriverInputControl
2021-03-09 11:03:45,355 DEBUG Binding PyCodelet lqr.driver_sophia_output/DriverOutputControl
2021-03-09 11:03:45,356 DEBUG Launching isaac core
2021-03-09 11:03:45,358 DEBUG Launching pycodelet threads
2021-03-09 11:03:45,358 DEBUG Launching command_generator/PyCodelet
2021-03-09 11:03:45,411 DEBUG Launching lqr.driver_sophia_input/DriverInputControl
2021-03-09 11:03:45,412 DEBUG Launching lqr.driver_sophia_output/DriverOutputControl



 --Inside Joint Controller-- 

------------------------------  Step IN   
------------------------------  Step OUT  


Inside START of Driver OUTPUT
Inside TICK Sophia Output
Inside TICK Sophia Output
Inside TICK Sophia Output
__  106, 64 and 430 Message received with following details  __
Tensor 106 [[[2241.  496.]]]
Tensor 64  [[[425.]]]
Tensor 430 [[[1931. 1713. 1644. 2202.]]]
	__END__
CHECK 1
Tensor Combined Looks Like [2241.  496.  425. 1931. 1713. 1644. 2202.]
CHECK 2
CHECK 3
Length of Quantities 7
Length of Tensor :  7
Quantities of 
VALUES OF COMPOSITE PROTO : [2241.  496.  425. 1931. 1713. 1644. 2202.]
Length of Quantities 7
Length of Tensor :  7
Quantities of 
VALUES OF COMPOSITE PROTO : [2241.  496.  425. 1931. 1713. 1644. 2202.]
Driver Position List [2241.  496.  740. 1931. 1713. 1644. 2202.]
Checking Size of tensor 
Message Received from Composite Proto [2241.  496.  740. 1931. 1713. 1644. 2202.]
Inside TICK Sophia Output
__  106, 64 and 430 Message received with following details  __
Tensor 106 [[[2241.  496.]]]
Tensor 64  [[[524.]]]
Tensor 430 [[[1930. 1714. 1643. 2203.]]]
	__END__
CHECK

Length of Quantities 7
Length of Tensor :  7
Quantities of 
VALUES OF COMPOSITE PROTO : [2246.  503.  730. 1931. 1714. 1643. 2201.]
Driver Position List [2241.  496.  740. 1931. 1713. 1644. 2202.]
Checking Size of tensor 
Message Received from Composite Proto [2241.  496.  740. 1931. 1713. 1644. 2202.]
Inside TICK Sophia Output
__  106, 64 and 430 Message received with following details  __
Tensor 106 [[[2246.  503.]]]
Tensor 64  [[[730.]]]
Tensor 430 [[[1931. 1714. 1643. 2202.]]]
	__END__
CHECK 1
Tensor Combined Looks Like [2246.  503.  730. 1931. 1714. 1643. 2202.]
CHECK 2
CHECK 3
Length of Quantities 7
Length of Tensor :  7
Quantities of 
VALUES OF COMPOSITE PROTO : [2246.  503.  730. 1931. 1714. 1643. 2202.]
Length of Quantities 7
Length of Tensor :  7
Quantities of 
VALUES OF COMPOSITE PROTO : [2246.  503.  730. 1931. 1714. 1643. 2202.]
Driver Position List [2241.  496.  740. 1931. 1713. 1644. 2202.]
Checking Size of tensor 
Message Received from Composite Proto [2241.  496.  740.

Inside TICK Sophia Output
__  106, 64 and 430 Message received with following details  __
Tensor 106 [[[2142.  504.]]]
Tensor 64  [[[729.]]]
Tensor 430 [[[1931. 1714. 1644. 2202.]]]
	__END__
CHECK 1
Tensor Combined Looks Like [2142.  504.  729. 1931. 1714. 1644. 2202.]
CHECK 2
CHECK 3
Length of Quantities 7
Length of Tensor :  7
Quantities of 
VALUES OF COMPOSITE PROTO : [2142.  504.  729. 1931. 1714. 1644. 2202.]
Length of Quantities 7
Length of Tensor :  7
Quantities of 
VALUES OF COMPOSITE PROTO : [2142.  504.  729. 1931. 1714. 1644. 2202.]
Driver Position List [1522.633  496.     740.    1931.    1713.    1644.    2202.   ]
Checking Size of tensor 
Message Received from Composite Proto [1522.633  496.     740.    1931.    1713.    1644.    2202.   ]
Inside TICK Sophia Output
__  106, 64 and 430 Message received with following details  __
Tensor 106 [[[2010.  504.]]]
Tensor 64  [[[730.]]]
Tensor 430 [[[1931. 1714. 1644. 2202.]]]
	__END__
CHECK 1
Tensor Combined Looks Like [2010.  50

...

Driver Position List [1522.633 1532.406  740.    1931.    1713.    1644.    2202.   ]
Checking Size of tensor 
Message Received from Composite Proto [1522.633 1532.406  740.    1931.    1713.    1644.    2202.   ]
Inside TICK Sophia Output
__  106, 64 and 430 Message received with following details  __
Tensor 106 [[[1537. 1520.]]]
Tensor 64  [[[729.]]]
Tensor 430 [[[1931. 1714. 1644. 2203.]]]
	__END__
CHECK 1
Tensor Combined Looks Like [1537. 1520.  729. 1931. 1714. 1644. 2203.]
CHECK 2
CHECK 3
Length of Quantities 7
Length of Tensor :  7
Quantities of 
VALUES OF COMPOSITE PROTO : [1537. 1520.  729. 1931. 1714. 1644. 2203.]
Length of Quantities 7
Length of Tensor :  7
Quantities of 
VALUES OF COMPOSITE PROTO : [1537. 1520.  729. 1931. 1714. 1644. 2203.]
Driver Position List [1522.633 1532.406  740.    1931.    1713.    1644.    2202.   ]
Checking Size of tensor 
Message Received from Composite Proto [1522.633 1532.406  740.    1931.    1713.    1644.    2202.   ]
Inside TICK Sophia Out

VALUES OF COMPOSITE PROTO : [1538. 1520.  729. 1931. 1714. 1644. 2203.]
Length of Quantities 7
Length of Tensor :  7
Quantities of 
VALUES OF COMPOSITE PROTO : [1538. 1520.  729. 1931. 1714. 1644. 2203.]
Driver Position List [1522.633 1532.406  740.    1931.    1713.    1644.    2202.   ]
Checking Size of tensor 
Message Received from Composite Proto [1522.633 1532.406  740.    1931.    1713.    1644.    2202.   ]
Inside TICK Sophia Output
__  106, 64 and 430 Message received with following details  __
Tensor 106 [[[1537. 1520.]]]
Tensor 64  [[[728.]]]
Tensor 430 [[[1931. 1714. 1644. 2203.]]]
	__END__
CHECK 1
Tensor Combined Looks Like [1537. 1520.  728. 1931. 1714. 1644. 2203.]
CHECK 2
CHECK 3
Length of Quantities 7
Length of Tensor :  7
Quantities of 
VALUES OF COMPOSITE PROTO : [1537. 1520.  728. 1931. 1714. 1644. 2203.]
Length of Quantities 7
Length of Tensor :  7
Quantities of 
VALUES OF COMPOSITE PROTO : [1537. 1520.  728. 1931. 1714. 1644. 2203.]
Driver Position List [1522.633 1

Length of Quantities 7
Length of Tensor :  7
Quantities of 
VALUES OF COMPOSITE PROTO : [1538. 1520. 1253. 1932. 1713. 1644. 2203.]
Driver Position List [1522.633 1532.406 2186.792 1931.    1713.    1644.    2202.   ]
Checking Size of tensor 
Message Received from Composite Proto [1522.633 1532.406 2186.792 1931.    1713.    1644.    2202.   ]
Inside TICK Sophia Output
__  106, 64 and 430 Message received with following details  __
Tensor 106 [[[1538. 1520.]]]
Tensor 64  [[[1391.]]]
Tensor 430 [[[1932. 1714. 1644. 2203.]]]
	__END__
CHECK 1
Tensor Combined Looks Like [1538. 1520. 1391. 1932. 1714. 1644. 2203.]
CHECK 2
CHECK 3
Length of Quantities 7
Length of Tensor :  7
Quantities of 
VALUES OF COMPOSITE PROTO : [1538. 1520. 1391. 1932. 1714. 1644. 2203.]
Length of Quantities 7
Length of Tensor :  7
Quantities of 
VALUES OF COMPOSITE PROTO : [1538. 1520. 1391. 1932. 1714. 1644. 2203.]
Driver Position List [1522.633 1532.406 2186.792 1931.    1713.    1644.    2202.   ]
Checking Size of 

Driver Position List [1522.633 1532.406 2186.792 1931.    1713.    1644.    2202.   ]
Checking Size of tensor 
Message Received from Composite Proto [1522.633 1532.406 2186.792 1931.    1713.    1644.    2202.   ]
Inside TICK Sophia Output
__  106, 64 and 430 Message received with following details  __
Tensor 106 [[[1537. 1519.]]]
Tensor 64  [[[2174.]]]
Tensor 430 [[[1933. 1714. 1644. 2203.]]]
	__END__
CHECK 1
Tensor Combined Looks Like [1537. 1519. 2174. 1933. 1714. 1644. 2203.]
CHECK 2
CHECK 3
Length of Quantities 7
Length of Tensor :  7
Quantities of 
VALUES OF COMPOSITE PROTO : [1537. 1519. 2174. 1933. 1714. 1644. 2203.]
Length of Quantities 7
Length of Tensor :  7
Quantities of 
VALUES OF COMPOSITE PROTO : [1537. 1519. 2174. 1933. 1714. 1644. 2203.]
Driver Position List [1522.633 1532.406 2186.792 1931.    1713.    1644.    2202.   ]
Checking Size of tensor 
Message Received from Composite Proto [1522.633 1532.406 2186.792 1931.    1713.    1644.    2202.   ]
Inside TICK Sophia Ou

Length of Quantities 7
Length of Tensor :  7
Quantities of 
VALUES OF COMPOSITE PROTO : [1538. 1520. 2174. 1933. 1714. 1644. 2204.]
Driver Position List [1522.633 1532.406 2186.792 1931.    1713.    1644.    2202.   ]
Checking Size of tensor 
Message Received from Composite Proto [1522.633 1532.406 2186.792 1931.    1713.    1644.    2202.   ]
Inside TICK Sophia Output
__  106, 64 and 430 Message received with following details  __
Tensor 106 [[[1538. 1520.]]]
Tensor 64  [[[2174.]]]
Tensor 430 [[[1933. 1714. 1644. 2203.]]]
	__END__
CHECK 1
Tensor Combined Looks Like [1538. 1520. 2174. 1933. 1714. 1644. 2203.]
CHECK 2
CHECK 3
Length of Quantities 7
Length of Tensor :  7
Quantities of 
VALUES OF COMPOSITE PROTO : [1538. 1520. 2174. 1933. 1714. 1644. 2203.]
Length of Quantities 7
Length of Tensor :  7
Quantities of 
VALUES OF COMPOSITE PROTO : [1538. 1520. 2174. 1933. 1714. 1644. 2203.]
Driver Position List [1522.633 1532.406 2186.792 1931.    1713.    1644.    2202.   ]
Checking Size of 

In [ ]:
app.stop()

In [ ]:
# app = Application(name="simple_joint_control_lqr_sophia_larm")

# # Load Applications
# app.load(filename="/home/mrobotics2/Desktop/isaac/sdk/packages/planner/apps/multi_joint_lqr_control.subgraph.json", prefix="lqr")
# lqr_interface = app.nodes["lqr.subgraph"]["interface"]

# #Dynamixel Interface
# # dynamixel_driver = app.nodes["lqr.dynamixel"]["driver"]

# #Driver Nodes - Sophia Larm
# # driver_sophia_input =  app.nodes["lqr.driver_sophia_input"]["DriverInputControl"]
# driver_sophia_output =  app.nodes["lqr.driver_sophia_output"]["DriverOutputControl"]


# # driver -> lqr
# #lqr -> driver
# # app.connect(driver_sophia_output, "state", lqr_interface, "joint_command")
# # app.connect(lqr_interface, "joint_state", driver_sophia_input, "command")


# # LQR configs
# app.nodes["lqr.kinematic_tree"]["KinematicTree"].config.kinematic_file = kinematic_file
# lqr_planner = app.nodes["lqr.local_plan"]["MultiJointLqrPlanner"]
# lqr_planner.config.speed_min = [-1.0] * len(joints)
# lqr_planner.config.speed_max = [1.0] * len(joints)
# lqr_planner.config.acceleration_min = [-1.0] * len(joints)
# lqr_planner.config.acceleration_max = [1.0] * len(joints)

# #Edges for Driver and Config
# # app.connect(driver_sophia_input, "state", dynamixel_driver, "command") # sophia_input ----> dynamixel_driver
# # app.connect(dynamixel_driver, "state", driver_sophia_output, "command") # dynamixel-driver -----> sophia_outptu
# # driver_sophia_output.config.joints = joints
# # driver_sophia_output.config.measure = "position"

# # add pycodelet JointControl for arm control
# joint_commander = app.add("command_generator").add(JointControl)
# joint_commander.config.joints = joints
# joint_commander.config.limits = [[0.2791845034, 4.4],[0.32, 5.95],[3.003534383, 3.882505374],[3.103243134, 4.017495683]]
# joint_commander.config.measure = "position"
# #Edges
# app.connect(joint_commander, "command", lqr_interface, "joint_target") #jointcommander ---> lqr
# app.connect(driver_sophia_output, "command", joint_commander, "state")  # driver_output ----> joint_commander


# app.nodes["command_generator"].add(name="JointControl", ctype=JointControl)
# app.nodes["lqr.driver_sopjoint_commandhia_input"].add(name="DriverInputControl", ctype=DriverInputControl)
# app.nodes["lqr.driver_sophia_output"].add(name="DriverOutputControl", ctype=DriverOutputControl)


# app.start()

UR10 with LQR planner in Omniverse Isaac Sim
======

Open the ur10_playground stage in Kit, start Robot Engine Bridge and hit Play before continuing

In [ ]:
# set kinematic file and get list of joints
kinematic_file = "apps/assets/kinematic_trees/ur10.kinematic.json"
joints = []
with open(kinematic_file,'r') as fd:
    kt = json.load(fd)
    for link in kt['links']:
        if 'motor' in link and link['motor']['type'] != 'constant':
            joints.append(link['name'])
print(joints)

In [ ]:
app = Application(name="simple_joint_control_lqr_ur10_sim")

# load subgraphs
app.load(filename="packages/planner/apps/multi_joint_lqr_control.subgraph.json", prefix="lqr")
app.load(filename="packages/navsim/apps/navsim_tcp.subgraph.json", prefix="simulation")
# edges
simulation_node = app.nodes["simulation.interface"]
lqr_interface = app.nodes["lqr.subgraph"]["interface"]
app.connect(simulation_node["output"], "joint_state", lqr_interface, "joint_state")
app.connect(lqr_interface, "joint_command", simulation_node["input"], "joint_position")

# configs
app.nodes["lqr.kinematic_tree"]["KinematicTree"].config.kinematic_file = kinematic_file
lqr_planner = app.nodes["lqr.local_plan"]["MultiJointLqrPlanner"]
lqr_planner.config.speed_min = [-1.0] * len(joints)
lqr_planner.config.speed_max = [1.0] * len(joints)
lqr_planner.config.acceleration_min = [-1.0] * len(joints)
lqr_planner.config.acceleration_max = [1.0] * len(joints)

# add pycodelet JointControl for arm control
joint_commander = app.add("command_generator").add(JointControl)
joint_commander.config.joints = joints
joint_commander.config.limits = [[-2*np.pi, 2*np.pi]] * len(joints)
joint_commander.config.measure = "position"
app.connect(joint_commander, "command", lqr_interface, "joint_target")
app.connect(simulation_node["output"], "joint_state", joint_commander, "state")

# add pycodelet JointPreview as a preview node between simulation and JointControl
# joint_preview = app.add("command_preview").add(JointPreview)
# joint_commander.config.joints = joints
# joint_commander.config.limits = [[-2*np.pi, 2*np.pi]] * len(joints)
# joint_commander.config.measure = "position"
# app.connect(joint_commander, "command", lqr_interface, "joint_target")
# app.connect(simulation_node["output"], "joint_state", joint_commander, "state")

# add pycodelet JointControl for finger control
finger_commander = app.add("finger_generator").add(JointControl)
finger_commander.config.joints = ["gripper"]
finger_commander.config.limits = [[0.0, 1.0]] * len(finger_commander.config.joints)
finger_commander.config.measure = "none"
app.connect(finger_commander, "command", simulation_node["input"], "io_command")
app.connect(simulation_node["output"], "io_state", finger_commander, "state")

app.start()

In [ ]:
# stop Isaac app
app.stop()

UR10 with RMP planner in Omniverse Isaac Sim
======

Open the ur10_playground stage in Kit, start Robot Engine Bridge and hit Play before continuing

In [ ]:
# set kinematic file and get list of joints
kinematic_file = "apps/assets/kinematic_trees/ur10.kinematic.json"
joints = []
with open(kinematic_file,'r') as fd:
    kt = json.load(fd)
    for link in kt['links']:
        if 'motor' in link and link['motor']['type'] != 'constant':
            joints.append(link['name'])
print(joints)

In [ ]:
app = Application(name="simple_joint_control_rmp_ur10_sim")

# load subgraphs
app.load(filename="packages/planner/apps/multi_joint_rmp_control.subgraph.json", prefix="rmp")
app.load(filename="packages/navsim/apps/navsim_tcp.subgraph.json", prefix="simulation")
# edges
simulation_node = app.nodes["simulation.interface"]
rmp_interface = app.nodes["rmp.subgraph"]["interface"]
app.connect(simulation_node["output"], "joint_state", rmp_interface, "joint_state")
app.connect(rmp_interface, "joint_command", simulation_node["input"], "joint_position")
# configs
app.nodes["rmp.kinematic_tree"]["KinematicTree"].config.kinematic_file = kinematic_file
rmp_planner = app.nodes["rmp.local_plan"]["MultiJointRmpPlanner"]
rmp_planner.config.rmpflow_config_file = "apps/assets/lula_assets/ur10_rmpflow_config.yaml"
rmp_planner.config.robot_description_file = "apps/assets/lula_assets/ur10_robot_description.yaml"
rmp_planner.config.robot_urdf = "apps/assets/lula_assets/ur10_robot.urdf"
rmp_planner.config.end_effector_frame = "tool"

# add pycodelet JointPositionControl
widget_node = app.add("command_generator")
joint_commander = widget_node.add(JointControl)
joint_commander.config.joints = joints
joint_commander.config.limits = [[-2*np.pi, 2*np.pi]] * len(joints)
joint_commander.config.measure = "position"
app.connect(joint_commander, "command", rmp_interface, "joint_target")
app.connect(simulation_node["output"], "joint_state", joint_commander, "state")
                            
app.start()

In [ ]:
# stop Isaac app
app.stop()

Kinova Jaco (gen2, 7 joints) Hardware
======

Install the KinoveJaco SDK in /opt/JACO2SDK (tested with v1.4.2) and connect to workstation via USB. Make sure the USB port has write permission

In [ ]:
kinematic_file = "apps/assets/kinematic_trees/kinova.kinematic.json"
joints = []
with open(kinematic_file,'r') as fd:
    kt = json.load(fd)
    for link in kt['links']:
        if 'motor' in link and link['motor']['type'] != 'constant':
            joints.append(link['name'])
print(joints)

In [ ]:
app = Application(name="simple_joint_control_kinova_real")

# load lqr subgraphcs
app.load(filename="packages/planner/apps/multi_joint_lqr_control.subgraph.json", prefix="lqr")
lqr_interface = app.nodes["lqr.subgraph"]["interface"]
# add kinova driver codelet
app.load_module("kinova_jaco")
- = app.add("driver").add(app.registry.isaac.kinova_jaco.KinovaJaco)
# edges
app.connect(driver, "arm_state", lqr_interface, "joint_state")
app.connect(lqr_interface, "joint_command", driver, "arm_command")
# configs
app.nodes["lqr.kinematic_tree"]["KinematicTree"].config.kinematic_file = kinematic_file
lqr_planner = app.nodes["lqr.local_plan"]["MultiJointLqrPlanner"]
lqr_planner.config.speed_min = [-0.5] * len(joints)
lqr_planner.config.speed_max = [0.5] * len(joints)
lqr_planner.config.acceleration_min = [-0.5] * len(joints)
lqr_planner.config.acceleration_max = [0.5] * len(joints)
app.nodes["lqr.controller"]["MultiJointController"].config.control_mode = "speed"
driver.config.kinematic_tree = "lqr.kinematic_tree"
driver.config.kinova_jaco_sdk_path = "/opt/JACO2SDK/API/"
driver.config.tick_period = "10ms"
driver.config.control_mode = "joint velocity"

# add pycodelet JointControl for arm control
joint_commander = app.add("command_generator").add(JointControl)
joint_commander.config.joints = joints
joint_commander.config.limits = [[-2*np.pi, 2*np.pi]] * len(joints)
joint_commander.config.measure = "position"
app.connect(joint_commander, "command", lqr_interface, "joint_target")
app.connect(driver, "arm_state", joint_commander, "state")

# add pycodelet JointControl for finger control
finger_commander = app.add("finger_generator").add(JointControl)
finger_commander.config.joints = ["finger1", "finger2", "finger3"]
finger_commander.config.limits = [[0.0, 1.0]] * len(finger_commander.config.joints)
finger_commander.config.measure = "none"

app.connect(finger_commander, "command", driver, "finger_command")
app.connect(driver, "finger_state", finger_commander, "state")

app.start()

In [ ]:
app.stop()